In [5]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
data_dir=r"/content/drive/MyDrive/team_audio_project"

In [8]:
def extract_features(file_path):
    try:
      audio, sample_rate=librosa.load(file_path,duration=3,offset=0.5)
      mfcc=librosa.feature.mfcc(y=audio,sr=sample_rate, n_mfcc=13)
      mfcc_scaled= np.mean(mfcc.T,axis=0)
      return mfcc_scaled
    except Exception as e:
      print(f"Error loading file:",file_path)
      return None

In [9]:
features=[]
labels=[]
for folder in os.listdir(data_dir):
  emotion = folder.split('_')[-1].lower()
  folder_path=os.path.join(data_dir,folder)
  for filename in os.listdir(folder_path):
    if filename.endswith('.wav'):
      file_path=os.path.join(folder_path,filename)
      mfcc=extract_features(file_path)
      if mfcc is not None:
        features.append(mfcc)
        labels.append(emotion)

In [10]:
x=np.array(features)
y=np.array(labels)

In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
model=RandomForestClassifier()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
print("Accuracy",accuracy_score(y_test,y_pred))
print("Classification Report",classification_report(y_test,y_pred))

Accuracy 0.75
Classification Report               precision    recall  f1-score   support

        hari       0.00      0.00      0.00         1
        joel       1.00      1.00      1.00         4
     praveen       1.00      1.00      1.00         1
     sukumar       0.50      0.50      0.50         2

    accuracy                           0.75         8
   macro avg       0.62      0.62      0.62         8
weighted avg       0.75      0.75      0.75         8



In [17]:
from google.colab import files
uploaded = files.upload()

Saving WhatsApp Audio 2025-07-18 at 12.07.10 PM (1).wav to WhatsApp Audio 2025-07-18 at 12.07.10 PM (1).wav


In [18]:
def predict_emotion(file_path):
  mfcc = extract_features(file_path)
  if mfcc is not None:
    return model.predict(mfcc.reshape(1,-1))[0]
  else:
   return "could not process the model"
for fname in uploaded.keys():
    print(f"Emotion predicted for {fname}: {predict_emotion(fname)}")

Emotion predicted for WhatsApp Audio 2025-07-18 at 12.07.10 PM (1).wav: sukumar
